In [1]:
import usb.core
import usb.util

EP_IN  = 0x82
EP_OUT = 0x2

In [2]:
# find FT600
dev = usb.core.find(idVendor=0x0403, idProduct=0x601e)
for config in dev:  
    # The device was getting "Err 16 busy" on my ubuntu
    for i in range(config.bNumInterfaces):
        if dev.is_kernel_driver_active(i):
            dev.detach_kernel_driver(i) 
dev.reset()

try:
    dev.set_configuration()
except:
    print("Already connected")
EP = []

for cfg in dev:
    for iface in cfg:
        for ep in iface:
            ep_dir = usb.util.endpoint_direction(ep.bEndpointAddress)
            ep_type = usb.util.endpoint_type(ep.bmAttributes)
            EP.append(ep)
            #print(ep)

In [3]:
EP

[<ENDPOINT 0x1: Bulk OUT>,
 <ENDPOINT 0x81: Interrupt IN>,
 <ENDPOINT 0x2: Bulk OUT>,
 <ENDPOINT 0x82: Bulk IN>]

In [8]:
k = dev.write(0x02, b'\x12\x34\x56\x78' * (4096//2) )

In [12]:
dev.read(EP[-1], 4096)  #<ENDPOINT 0x1: Bulk OUT>,

USBTimeoutError: [Errno 110] Operation timed out

# Setup FTDI side

In [1]:
import sys;
import select;
import socket;
import time;
import os;
import random;
from time import sleep;

In [2]:
import ftd3xx
import sys
import ftd3xx._ftd3xx_linux as _ft

Loading linux facilities


In [3]:
ftd3xx.__file__

'/home/pi/.local/lib/python3.7/site-packages/ftd3xx/__init__.py'

In [4]:
numDevices = ftd3xx.createDeviceInfoList()
if (numDevices == 0):
    print("ERROR: No FTD3XX device is detected.");
else:
    print(numDevices," Device")
devList = ftd3xx.getDeviceInfoList()
print(devList)
# Just open the first device (index 0)
devIndex = 0;
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);  

1  Device


In [5]:
cfg = D3XX.getChipConfiguration();
print("CLK: ",cfg.FIFOClock)
print("Mode: ",cfg.FIFOMode)
print("FlashEEPROMDetection: ",cfg.FlashEEPROMDetection)
VP = D3XX.getVIDPID()
print("VID/PID:",hex(VP[0]), hex(VP[1]))

CLK:  1
Mode:  0
FlashEEPROMDetection:  16
VID/PID: 0x60 0x1e


In [20]:
# Reset the FT600 to make sure starting fresh with nothing in FIFOs
D3XX.resetDevicePort(); # Flush
D3XX.close();

In [7]:

D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);

# check if USB3 or USB2
devDesc = D3XX.getDeviceDescriptor();
bUSB3 = devDesc.bcdUSB >= 0x300;

# validate chip configuration
cfg = D3XX.getChipConfiguration();
cfg.FIFOClock

1

In [8]:
D3XX.setChipConfiguration(cfg);

In [9]:
desired_freq = 1;# 1=66 MHz, 0=100 MHz
if ( cfg.FIFOClock != desired_freq ):
    print("Changing FT600 Frequency");
    cfg.FIFOClock = desired_freq;
    D3XX.setChipConfiguration(cfg);
D3XX.resetDevicePort(); # Flush
D3XX.close();

In [10]:
numDevices = ftd3xx.createDeviceInfoList()
if (numDevices == 0):
    print("ERROR: No FTD3XX device is detected.");
else:
    print(numDevices," Device")
devList = ftd3xx.getDeviceInfoList()
print(devList)
devIndex = 0;
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);


1  Device


In [11]:
cfg = D3XX.getChipConfiguration();
cfg.FIFOClock

cfg = D3XX.getChipConfiguration();
print("\tVendorID = %#06x" % cfg.VendorID)
print("\tProductID = %#06x" % cfg.ProductID)
cfg.FIFOMode,D3XX.getFirmwareVersion()

	VendorID = 0x0403
	ProductID = 0x601e


(0, 265)

In [12]:
import ctypes

In [13]:
conf = _ft.FT_TRANSFER_CONF();
conf.wStructSize = ctypes.sizeof(_ft.FT_TRANSFER_CONF);
conf.pipe[_ft.FT_PIPE_DIR_IN].fPipeNotUsed = True;
conf.pipe[_ft.FT_PIPE_DIR_OUT].fPipeNotUsed = True;
conf.pipe.fReserved = False;
conf.pipe.fKeepDeviceSideBufferAfterReopen = False;
for i in range(4):
    ftd3xx.setTransferParams(conf, i);

In [14]:
print(sys.platform)
def ReadPipe(D3XX, pipe, size):

    if sys.platform == 'linux':
        pipe -= 0x82
    print("Pipe, ",pipe)
    return D3XX.readPipeEx(pipe, size)

def CancelPipe(D3XX, pipe):

    if sys.platform == 'linux':
        return D3XX.flushPipe(pipe)

    return D3XX.abortPipe(pipe)

linux


In [15]:
# Call before FT_Create when non-transfer functions will be called
# Only needed for RevA chip (Firmware 1.0.2)
# Not necessary starting RevB chip (Firmware 1.0.9)
conf = _ft.FT_TRANSFER_CONF();
conf.wStructSize = ctypes.sizeof(_ft.FT_TRANSFER_CONF);
conf.pipe[_ft.FT_PIPE_DIR_IN].fPipeNotUsed = True;
conf.pipe[_ft.FT_PIPE_DIR_OUT].fPipeNotUsed = True;
conf.pipe.fReserved = False;
conf.pipe.fKeepDeviceSideBufferAfterReopen = False;
for i in range(4):
    ftd3xx.setTransferParams(conf, i);


In [16]:
D3XX.getLastError()

0

In [17]:
numChannels = [4, 2, 1, 0, 0]
numChannels = numChannels[cfg.ChannelConfig]
numChannels

1

In [19]:
ReadPipe(D3XX,  0x82, 4096)

Pipe,  0


{'bytesTransferred': 4096,
 'bytes': b'\x06\xff\x07\xff\x0c\xff\r\xff\x0e\xff\x0f\xff\x10\xff\x11\xff\x12\xff\x13\xff\x18\xff\x19\xff\x1a\xff\x1b\xff\x14\xff\x15\xff\x16\xff\x17\xff\x1c\xff\x1d\xff\x1e\xff\x1f\xff \xff!\xff"\xff#\xff(\xff)\xff*\xff+\xff$\xff%\xff&\xff\'\xff,\xff-\xff.\xff/\xff0\xff1\xff2\xff3\xff8\xff9\xff:\xff;\xff4\xff5\xff6\xff7\xff<\xff=\xff>\xff?\xff@\xffA\xffB\xffC\xffH\xffI\xffJ\xffK\xffD\xffE\xffF\xffG\xffL\xffM\xffN\xffO\xffP\xffQ\xffR\xffS\xffX\xffY\xffZ\xff[\xffT\xffU\xffV\xffW\xff\\\xff]\xff^\xff_\xff`\xffa\xffb\xffc\xffh\xffi\xffj\xffk\xffd\xffe\xfff\xffg\xffl\xffm\xffn\xffo\xffp\xffq\xffr\xffs\xffx\xffy\xffz\xff{\xfft\xffu\xffv\xffw\xff|\xff}\xff~\xff\x7f\xff\x80\xff\x81\xff\x82\xff\x83\xff\x88\xff\x89\xff\x8a\xff\x8b\xff\x84\xff\x85\xff\x86\xff\x87\xff\x8c\xff\x8d\xff\x8e\xff\x8f\xff\x90\xff\x91\xff\x92\xff\x93\xff\x98\xff\x99\xff\x9a\xff\x9b\xff\x94\xff\x95\xff\x96\xff\x97\xff\x9c\xff\x9d\xff\x9e\xff\x9f\xff\xa0\xff\xa1\xff\xa2\xff\xa3\xff\xa8\xff\xa9\x

In [22]:
D3XX.resetDevicePort(); # Flush
D3XX.close();